In [ ]:
# default_exp gradient

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# gradient - Approximating the gradient




A collection of classes and functions used to approximate the gradient.
***
## Background

Spall's simultaneous perturbation stochastic approximation (SPSA) of the gradient provides an efficient means to approximate the gradient of high-dimensional models, even when only noisy evaluations of the objective function are available. This is in constrast to more typical applications of stochastic gradient descent, where the noisiness of the gradient comes not from the objective function itself, but rather from evaluating the gradient on subsets of the data. 

### Approximating the gradient with SPSA 

The general idea of SPSA is reasonably straightforward. Given a step size $c_t$ and a vector of perturbations $\delta$, we first generate forward and backward perturbations all model parameters simultaneously

$$\theta^+ = \theta + c_t \delta$$
$$\theta^- = \theta - c_t \delta$$

The perturbation, $\delta$ is often sampled from a shifted and rescaled Bernoulli distribution as follows:

$$b_1, b_2,..., b_m \sim Bernoulli(p=.5)$$
$$\delta_i = 2b_i -1$$

where $\delta_i$ is the direction in which the $i$-th model parameter will be moved in the forward perturbation.

We then evaluate the cost function $F(\theta, X)$ at the two perturbed parameters

$$y^+ = F(\theta^+, X)$$
$$y^- = F(\theta^-, X)$$

The gradient is approximated as the slope of the line between the points $(\theta^+, y^+)$ and $(\theta^-, y^-)$:

$$\hat{g}= \frac{y^+-y^-}{\theta^+ - \theta^-}= \frac{y^+-y^-}{2 c_t \delta}$$

A major advantage of this approximation is that in its simplest form, only two evaluations of the cost function are required, regardless of the dimensionality of the model. This is in constrast to the [finite-differences approximation]() which requires each model parameter be perturbed separately.

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export 
import numpy
import scipy
from abc import ABC, abstractmethod 

In [ ]:
#export

class GradientBase(ABC):
    """A helper class that provides a standard means to create
    classes to provide gradients or their approximations to GradientDescent."""
    @abstractmethod
    #This is the workhorse of the class
    def evaluate(self): pass



***

```GradientDescent``` must be passed an object with a method called ```.evaluate()```. This should store as an attribute the cost function to be evaluated and take the following inputs:

1. theta - A 1-D numpy array of model parameters
2. c_k - A step size that may be used in the gradient evaluation
3. gradient_reps - The number of times to evaluate the gradient (multiple evaluations will be averaged)
4. update_rvs - Whether regenerated random variables stored in the cost function after each gradient evaluation

It should return a vector of the same length as ```theta``` containing an estimate of the cost function's gradient at ```theta```.

Any approach to gradient evaluation will require the first argument, ```theta```. The latter three are only necessary when using an approximation of the gradient. 


In [ ]:
#export
class SPSAGradient(GradientBase):
    """A class for computing the SPSA gradient estimate."""
    def __init__(self, param_subsets=None,fraction=None, cost=None):
        self.cost=cost
        self.param_subsets=param_subsets
        if self.param_subsets is not None:
            self.param_subsets=numpy.array(self.param_subsets)
            self.subsets=set(list(param_subsets))
        
    def set_cost(self, cost):
        self.cost=cost
        
    def evaluate(self, theta, c_k, gradient_reps=1, update_rvs=False):
        """Inputs
        1. theta - A 1-D numpy array of model parameters
        2. c_k - A step size that may be used in the gradient evaluation
        3. gradient_reps - The number of times to evaluate the gradient 
            (multiple evaluations will be averaged)
        4. update_rvs - Whether regenerated random variables stored in 
            the cost function after each gradient evaluation
            
            Returns an array gradient estimates the same size as theta

        """

#         assert len(theta)==len(self.)
        #If no subsets were defined, then now we'll define all model parameters as one set
        assert self.cost is not None
        
        if self.param_subsets is None:
            self.param_subsets=numpy.zeros(theta.shape[0])
            self.subsets=set(list(self.param_subsets))
        #evaluate the gradient separately for different groups of parameters
        grad_list=[]
        for rep in range(gradient_reps):
            if update_rvs==True:     #Regenerate the random numbers in the cost with each gradient
                self.cost.sample_rvs()
            ghat=numpy.zeros(theta.shape)
            for s in self.subsets:
                
                param_filter=self.param_subsets==s

                ghat+=self.SPSA( theta, c_k, param_filter)
            grad_list.append(ghat)
        if gradient_reps==1:
            return grad_list[0]
        else: #We need to average
#             print (grad_list)
#             print ( numpy.mean(grad_list,0))
#             print (jabber)
            return numpy.mean(grad_list,0)
        
    def SPSA(self, theta, ck, param_ind):
        """ Inputs:
            cost - a function that takes model parameters and data as inputs
                    and returns a single float
            data - the data the model is being fit to
            theta - a set model parameters
            ck - the step size to be used during perturbation of the model parameters

            Outputs:
            An estimate of the gradient
        """
        #Draw the perturbation

        delta=2.*scipy.stats.bernoulli.rvs(p=.5,size=theta.shape[0])-1.
        #hold delta constant for the parameters not under consideration
        delta[~param_ind]=0.
        #Perturb the parameters forwards and backwards
        thetaplus=theta+ck*delta
        thetaminus=theta-ck*delta

        #Evaluate the objective after the perturbations
        yplus=self.cost.evaluate(thetaplus)
        yminus=self.cost.evaluate(thetaminus)

        #Compute the slope across the perturbation

        ghat=(yplus-yminus)/(2*ck*delta)

        ghat[~param_ind]=0
        return ghat

***

The `SPSAGradient` class is used by `GradientDescent` to approximate the gradient of an objective function, which can then be used to update model parameters.

This takes two arguments, both of which are optional:

1. ```param_subsets``` (optional) - A list or array of labels that defines groups of parameters. For example, \[0,0,0,1,1,1] defines the first three model parameters as group 0 and the last three as belong to group 1.  

2. ```cost``` (optional) - The cost function used in the gradient evaluation. When passing an instance of the `SPSAGradient` class to the `GradientDescent` optimizer, this should be left undefined. The `GradientDescent` object will automatically add the cost function being optimized to the `SPSAGradient` if its cost function has not been defined. 



#### Perturbing subset of parameters

In some models, it might be desirable to evaluate the gradient separately for different subsets of parameters. For example, in variational inference, the means of the posterior approximation have a much stronger impact on the loss function than the standard deviations do. In that case, perturbing all parameters at once is likely to pick up the impact of perturbing the means on the gradient, but perhaps not the standard deviations.

The ```param_labels``` option permits to the gradient approximation to be evaluated separately for subsets of parameters. If, for example. ```param_labels=[0,0,0,1,1,1]```, then the gradient will be approximated in two steps. The gradient will be estimated first for the three first parameters, perturbing them while holding the other parameters constant. Then the parameters labelled ```1``` will be perturbed, while all others are held constant. The cost of doing this is the number of cost function evaluations increases from $2$ to $2n$, where is $n$ number of parameter subset to be evaluated separately. 

#### Averaging multiple gradient approximations

By default calling ```evaluate``` approximates the gradient from a single forward and backward perturbation. The argument ```gradient_reps``` can instead be set to an integer value greater than 1, to instead return the average of multiple gradient evaluations. If ```gradient_reps``` is set to $r$, ```evaluate``` will return the average of $r$ gradient approximations. This may lead to faster convergences.